均线 三

我是格子衫小C（微信公众号：格子衫小C），今天开始，我们一起来研究均线指标（Indicator）。

均线是最常见的一种指标，一般交易软件里都能看到。

均线，顾名思义，就是用各种加权算法算出历史平均价格。

因为均线有明显的滞后性（譬如用前5日价格算出来的平均数，高点或低点都一定是滞后的），所以一般叫 移动平均线，也就是 Moving Averages ，简称 MA。

MA里，平均数的算法有很多种，因此MA也有很多种，其中，最简单的就是算术平均数算出来的，一般叫 MA 或 SMA。

In [ ]:
import pandas as pd
from datetime import datetime
import trdb2py

isStaticImg = False
width = 960
height = 768

pd.options.display.max_columns = None
pd.options.display.max_rows = None

trdb2cfg = trdb2py.loadConfig('./trdb2.yaml')

In [ ]:
# 具体基金
# asset = 'jrj.510310'
asset = 'jqdata.000300_XSHG|1d'

# 起始时间，0表示从最开始算起
# tsStart = 0
tsStart = int(trdb2py.str2timestamp('2013-05-01', '%Y-%m-%d'))

# 结束时间，-1表示到现在为止
# tsEnd = -1
tsEnd = int(trdb2py.str2timestamp('2020-09-30', '%Y-%m-%d'))

# 初始资金池
paramsinit = trdb2py.trading2_pb2.InitParams(
    money=10000,
)

# 买入参数，用全部的钱来买入（也就是复利）
paramsbuy = trdb2py.trading2_pb2.BuyParams(
    perHandMoney=1,
)

# 卖出参数，全部卖出
paramssell = trdb2py.trading2_pb2.SellParams(
    perVolume=1,
)

In [ ]:
asset = 'jqdata.000300_XSHG|1d'

# 起始时间，0表示从最开始算起
# tsStart = 0
tsStart = int(trdb2py.str2timestamp('2013-05-01', '%Y-%m-%d'))

# 结束时间，-1表示到现在为止
# tsEnd = -1
tsEnd = int(trdb2py.str2timestamp('2020-09-30', '%Y-%m-%d'))

# baseline    
s0 = trdb2py.trading2_pb2.Strategy(
    name="normal",
    asset=trdb2py.str2asset(asset),         
)
        
buy0 = trdb2py.trading2_pb2.CtrlCondition(
    name='buyandhold',
)

paramsbuy = trdb2py.trading2_pb2.BuyParams(
    perHandMoney=1,
)

paramsinit = trdb2py.trading2_pb2.InitParams(
    money=10000,
)

s0.buy.extend([buy0])
s0.paramsBuy.CopyFrom(paramsbuy)
s0.paramsInit.CopyFrom(paramsinit)        
p0 = trdb2py.trading2_pb2.SimTradingParams(
    assets=[trdb2py.str2asset(asset)],
    startTs=tsStart,
    endTs=tsEnd,
    strategies=[s0],
    title='沪深300指数',
)  

pnlBaseline = trdb2py.simTrading(trdb2cfg, p0)
trdb2py.showPNL(pnlBaseline, toImg=isStaticImg, width=width, height=height)

我们还是拿沪深300指数，2013年5月1日到2020年9月30日的日线数据来做例子。

接下来，测一下均线

In [ ]:
lstparams = []

asset = 'jqdata.000300_XSHG|1d'

# 起始时间，0表示从最开始算起
# tsStart = 0
tsStart = int(trdb2py.str2timestamp('2013-05-01', '%Y-%m-%d'))

# 结束时间，-1表示到现在为止
# tsEnd = -1
tsEnd = int(trdb2py.str2timestamp('2020-09-30', '%Y-%m-%d'))


for ema0 in range(2, 121):
    for ema1 in range(ema0 + 1, 121):
        buy0 = trdb2py.trading2_pb2.CtrlCondition(
            name='indicatordp',
            operators=['upcross'],
            strVals=['ta-sma.{}'.format(ema0), 'ta-sma.{}'.format(ema1)],
        )

        buy1 = trdb2py.trading2_pb2.CtrlCondition(
            name='waittostart',
            vals=[ema0],
        )
        
#         buy2 = trdb2py.trading2_pb2.CtrlCondition(
#             name='indicatorsp',
#             operators=['up'],
#             strVals=['ta-sma.{}'.format(ema0)],
#         )
        
        buy3 = trdb2py.trading2_pb2.CtrlCondition(
            name='waittostart',
            vals=[ema1],
        )        

        sell0 = trdb2py.trading2_pb2.CtrlCondition(
            name='indicatordp',
            operators=['downcross'],
            strVals=['ta-sma.{}'.format(ema0), 'ta-sma.{}'.format(ema1)],
        )
        
#         sell1 = trdb2py.trading2_pb2.CtrlCondition(
#             name='indicatorsp',
#             operators=['down'],
#             strVals=['ta-sma.{}'.format(ema0)],
#         )

        s0 = trdb2py.trading2_pb2.Strategy(
            name="normal",
            asset=trdb2py.str2asset(asset),
        )

        s0.buy.extend([buy0, buy1, buy3])
        s0.sell.extend([sell0])
        s0.paramsBuy.CopyFrom(paramsbuy)
        s0.paramsSell.CopyFrom(paramssell) 
        s0.paramsInit.CopyFrom(paramsinit)        
        lstparams.append(trdb2py.trading2_pb2.SimTradingParams(
            assets=[trdb2py.str2asset(asset)],
            startTs=tsStart,
            endTs=tsEnd,
            strategies=[s0],
            title='sma{}&sma{}'.format(ema0, ema1),
        ))
        
# len(lstparams)
lstpnlmix = trdb2py.simTradings(trdb2cfg, lstparams, ignoreTotalReturn=2.45)        

trdb2py.showPNLs(lstpnlmix + [pnlBaseline], toImg=isStaticImg, width=width, height=height)

In [ ]:
dfpnl = trdb2py.buildPNLReport(lstpnlmix + [pnlBaseline])

# dfpnl1 = dfpnl[dfpnl['totalReturns'] >= 2]

dfpnl[['title', 'maxDrawdown', 'maxDrawdownStart', 'maxDrawdownEnd', 'totalReturns', 'sharpe', 'annualizedReturns', 'annualizedVolatility', 'variance']].sort_values(by='totalReturns', ascending=False)

In [ ]:
lstparams = []

# 具体基金
# asset = 'jrj.510310'
asset = 'jqdata.000300_XSHG|1d'

# 起始时间，0表示从最开始算起
# tsStart = 0
tsStart = int(trdb2py.str2timestamp('2013-05-01', '%Y-%m-%d'))

# 结束时间，-1表示到现在为止
# tsEnd = -1
tsEnd = int(trdb2py.str2timestamp('2020-09-30', '%Y-%m-%d'))

for ema in range(2, 241):    
    buy0 = trdb2py.trading2_pb2.CtrlCondition(
        name='indicatorsp',
        operators=['upcross'],
        strVals=['ta-sma.{}'.format(ema)],
    )
    
    buy1 = trdb2py.trading2_pb2.CtrlCondition(
        name='waittostart',
        vals=[ema],
    )    
            
    sell0 = trdb2py.trading2_pb2.CtrlCondition(
        name='indicatorsp',
        operators=['downcross'],
        strVals=['ta-sma.{}'.format(ema)],
    )
    
    s0 = trdb2py.trading2_pb2.Strategy(
        name="normal",
        asset=trdb2py.str2asset(asset),
    )

    s0.buy.extend([buy0, buy1])
    s0.sell.extend([sell0])
    s0.paramsBuy.CopyFrom(paramsbuy)
    s0.paramsSell.CopyFrom(paramssell) 
    s0.paramsInit.CopyFrom(paramsinit)        
    lstparams.append(trdb2py.trading2_pb2.SimTradingParams(
        assets=[trdb2py.str2asset(asset)],
        startTs=tsStart,
        endTs=tsEnd,
        strategies=[s0],
        title='sma.{}'.format(ema),
    ))
 
lstpnlmix0 = trdb2py.simTradings(trdb2cfg, lstparams, ignoreTotalReturn=2.3)

trdb2py.showPNLs(lstpnlmix + lstpnlmix0 + [pnlBaseline], toImg=isStaticImg, width=width, height=height)

In [ ]:
dfpnl = trdb2py.buildPNLReport(lstpnlmix + lstpnlmix0 + [pnlBaseline])

# dfpnl1 = dfpnl[dfpnl['totalReturns'] >= 2]

dfpnl[['title', 'maxDrawdown', 'maxDrawdownStart', 'maxDrawdownEnd', 'totalReturns', 'sharpe', 'annualizedReturns', 'annualizedVolatility', 'variance']].sort_values(by='totalReturns', ascending=False)

In [ ]:
lstparams = []

lstsma = [57, 24, 63, 116, 42, 60]

# 具体基金
# asset = 'jrj.510310'
asset = 'jqdata.000300_XSHG|1d'

# 起始时间，0表示从最开始算起
# tsStart = 0
tsStart = int(trdb2py.str2timestamp('2013-05-01', '%Y-%m-%d'))

# 结束时间，-1表示到现在为止
# tsEnd = -1
tsEnd = int(trdb2py.str2timestamp('2020-09-30', '%Y-%m-%d'))

for ema in lstsma:    
    buy0 = trdb2py.trading2_pb2.CtrlCondition(
        name='indicatorsp',
        operators=['upcross'],
        strVals=['ta-sma.{}'.format(ema)],
    )
    
    buy1 = trdb2py.trading2_pb2.CtrlCondition(
        name='waittostart',
        vals=[ema],
    )    
            
    sell0 = trdb2py.trading2_pb2.CtrlCondition(
        name='indicatorsp',
        operators=['downcross'],
        strVals=['ta-sma.{}'.format(ema)],
    )
    
    s0 = trdb2py.trading2_pb2.Strategy(
        name="normal",
        asset=trdb2py.str2asset(asset),
    )

    s0.buy.extend([buy0, buy1])
    s0.sell.extend([sell0])
    s0.paramsBuy.CopyFrom(paramsbuy)
    s0.paramsSell.CopyFrom(paramssell) 
    s0.paramsInit.CopyFrom(paramsinit)        
    lstparams.append(trdb2py.trading2_pb2.SimTradingParams(
        assets=[trdb2py.str2asset(asset)],
        startTs=tsStart,
        endTs=tsEnd,
        strategies=[s0],
        title='sma.{}'.format(ema),
    ))
 
lstpnlmix1 = trdb2py.simTradings(trdb2cfg, lstparams, ignoreTotalReturn=2.3)

trdb2py.showPNLs(lstpnlmix + lstpnlmix1 + [pnlBaseline], toImg=isStaticImg, width=width, height=height)

In [ ]:
asset = 'jqdata.000300_XSHG|1d'

# 起始时间，0表示从最开始算起
tsStart = 0
# tsStart = int(trdb2py.str2timestamp('2013-05-01', '%Y-%m-%d'))

# 结束时间，-1表示到现在为止
tsEnd = -1
# tsEnd = int(trdb2py.str2timestamp('2020-09-30', '%Y-%m-%d'))

# baseline    
s0 = trdb2py.trading2_pb2.Strategy(
    name="normal",
    asset=trdb2py.str2asset(asset),         
)
        
buy0 = trdb2py.trading2_pb2.CtrlCondition(
    name='buyandhold',
)

paramsbuy = trdb2py.trading2_pb2.BuyParams(
    perHandMoney=1,
)

paramsinit = trdb2py.trading2_pb2.InitParams(
    money=10000,
)

s0.buy.extend([buy0])
s0.paramsBuy.CopyFrom(paramsbuy)
s0.paramsInit.CopyFrom(paramsinit)        
p0 = trdb2py.trading2_pb2.SimTradingParams(
    assets=[trdb2py.str2asset(asset)],
    startTs=tsStart,
    endTs=tsEnd,
    strategies=[s0],
    title='沪深300指数',
)  

pnlBaselineF = trdb2py.simTrading(trdb2cfg, p0)
trdb2py.showPNL(pnlBaselineF, toImg=isStaticImg, width=width, height=height)

In [ ]:
lstparams = []

lstems = [2, 35, 42, 60, 116, 135, 197, 240]
# 起始时间，0表示从最开始算起
tsStart = 0
# tsStart = int(trdb2py.str2timestamp('2013-05-01', '%Y-%m-%d'))

# 结束时间，-1表示到现在为止
# tsEnd = -1
# tsEnd = int(trdb2py.str2timestamp('2020-09-30', '%Y-%m-%d'))
tsEnd = int(trdb2py.str2timestamp('2020-12-31', '%Y-%m-%d'))

for ema in lstems:    
    buy0 = trdb2py.trading2_pb2.CtrlCondition(
        name='indicatorsp',
        operators=['upcross'],
        strVals=['ta-sma.{}'.format(ema)],
    )
    
    buy1 = trdb2py.trading2_pb2.CtrlCondition(
        name='waittostart',
        vals=[ema],
    )    
            
    sell0 = trdb2py.trading2_pb2.CtrlCondition(
        name='indicatorsp',
        operators=['downcross'],
        strVals=['ta-sma.{}'.format(ema)],
    )
    
    s0 = trdb2py.trading2_pb2.Strategy(
        name="normal",
        asset=trdb2py.str2asset(asset),
    )

    s0.buy.extend([buy0, buy1])
    s0.sell.extend([sell0])
    s0.paramsBuy.CopyFrom(paramsbuy)
    s0.paramsSell.CopyFrom(paramssell) 
    s0.paramsInit.CopyFrom(paramsinit)        
    lstparams.append(trdb2py.trading2_pb2.SimTradingParams(
        assets=[trdb2py.str2asset(asset)],
        startTs=tsStart,
        endTs=tsEnd,
        strategies=[s0],
        title='sma.{}'.format(ema),
    ))
 
lstpnlmixF0 = trdb2py.simTradings(trdb2cfg, lstparams, ignoreTotalReturn=2.3)

trdb2py.showPNLs(lstpnlmixF0 + [pnlBaselineF], toImg=isStaticImg, width=width, height=height)

In [ ]:
lstparams = []

asset = 'jqdata.000300_XSHG|1d'

# 起始时间，0表示从最开始算起
tsStart = 0
# tsStart = int(trdb2py.str2timestamp('2013-05-01', '%Y-%m-%d'))

# 结束时间，-1表示到现在为止
# tsEnd = -1
tsEnd = int(trdb2py.str2timestamp('2020-12-31', '%Y-%m-%d'))

lstema0 = [5, 20, 30, 2, 8]
lstema1 = [57, 24, 59, 116, 60]


for i in range(0, len(lstema0)):
    buy0 = trdb2py.trading2_pb2.CtrlCondition(
        name='indicatordp',
        operators=['upcross'],
        strVals=['ta-sma.{}'.format(lstema0[i]), 'ta-sma.{}'.format(lstema1[i])],
    )

    buy1 = trdb2py.trading2_pb2.CtrlCondition(
        name='waittostart',
        vals=[lstema0[i]],
    )

#         buy2 = trdb2py.trading2_pb2.CtrlCondition(
#             name='indicatorsp',
#             operators=['up'],
#             strVals=['ta-sma.{}'.format(ema0)],
#         )

    buy3 = trdb2py.trading2_pb2.CtrlCondition(
        name='waittostart',
        vals=[lstema1[i]],
    )        

    sell0 = trdb2py.trading2_pb2.CtrlCondition(
        name='indicatordp',
        operators=['downcross'],
        strVals=['ta-sma.{}'.format(lstema0[i]), 'ta-sma.{}'.format(lstema1[i])],
    )

#         sell1 = trdb2py.trading2_pb2.CtrlCondition(
#             name='indicatorsp',
#             operators=['down'],
#             strVals=['ta-sma.{}'.format(ema0)],
#         )

    s0 = trdb2py.trading2_pb2.Strategy(
        name="normal",
        asset=trdb2py.str2asset(asset),
    )

    s0.buy.extend([buy0, buy1, buy3])
    s0.sell.extend([sell0])
    s0.paramsBuy.CopyFrom(paramsbuy)
    s0.paramsSell.CopyFrom(paramssell) 
    s0.paramsInit.CopyFrom(paramsinit)        
    lstparams.append(trdb2py.trading2_pb2.SimTradingParams(
        assets=[trdb2py.str2asset(asset)],
        startTs=tsStart,
        endTs=tsEnd,
        strategies=[s0],
        title='sma{}&sma{}'.format(lstema0[i], lstema1[i]),
    ))
        
lstpnlmixF = trdb2py.simTradings(trdb2cfg, lstparams, ignoreTotalReturn=3)        

trdb2py.showPNLs(lstpnlmixF + lstpnlmixF0 + [pnlBaselineF], toImg=isStaticImg, width=width, height=height)

In [ ]:
dfpnl = trdb2py.buildPNLReport(lstpnlmixF + lstpnlmixF0 + [pnlBaselineF])

# dfpnl1 = dfpnl[dfpnl['totalReturns'] >= 2]

dfpnl[['title', 'maxDrawdown', 'maxDrawdownStart', 'maxDrawdownEnd', 'totalReturns', 'sharpe', 'annualizedReturns', 'annualizedVolatility', 'variance']].sort_values(by='totalReturns', ascending=False)